<a href="https://colab.research.google.com/github/Yash4-bee/Bank-Loan-Repayment-Prediction-Model-using-Machine-Learning/blob/main/Churn_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Project Title:
    # Predicting Customer churn using Machine Learning

# Buisness problem:
    # In Netflix every month some users cancel their subscription and leave the platform, which is called as churn.
    # The goal is to predict who might leave next, so that Netflix can hold them. by offering discounts or personalised engagement.

# Datasets: We have a 4 Datasets given,
    # 1. Customers - This have the basic details of customers info like name, status(active/inactive), region etc.
    # 2. Churn - It tell us the Customer who left us with dates.
    # 3. subscription - It carries the detail about their purchased plan (start and end date)
    # 4. Transaction - In this we have detail of customer spendings over the time.

In [2]:
# Workflow
    # 1. Combining all the datasets into one big table.
    # 2. Feature Engineering.
    # 3. Defining Independent and Dependent variables.

In [17]:
# Importing all the required libraries

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [18]:
# Loading all the Datasets

Customers = pd.read_csv('/content/drive/MyDrive/Projects/ML/Customers.csv')
Churn = pd.read_csv('/content/drive/MyDrive/Projects/ML/Churn.csv')
Subscription = pd.read_csv('/content/drive/MyDrive/Projects/ML/Subscriptions.csv')
Transaction = pd.read_csv('/content/drive/MyDrive/Projects/ML/Transactions.csv')

print(Customers.columns)
print(Churn.columns)
print(Subscription.columns)
print(Transaction.columns)

Index(['CustomerID', 'FirstName', 'LastName', 'Email', 'PhoneNumber',
       'JoinDate', 'Status', 'Region'],
      dtype='object')
Index(['ChurnID', 'CustomerID', 'ChurnDate', 'Reason'], dtype='object')
Index(['SubscriptionID', 'CustomerID', 'StartDate', 'EndDate', 'PlanType'], dtype='object')
Index(['TransactionID', 'CustomerID', 'TransactionDate', 'Amount',
       'TransactionType'],
      dtype='object')


In [19]:
# Merging all the datasets into one

df= Customers.merge(Subscription ,on='CustomerID', how = 'left')
df.head(10)

,CustomerID,FirstName,LastName,Email,PhoneNumber,JoinDate,Status,Region,SubscriptionID,StartDate,EndDate,PlanType
0,1,John,Doe,john.doe@example.com,123-456-7890,10-01-2022,Active,North America,1.0,10-01-2022,09-01-2023,Annual
1,1,John,Doe,john.doe@example.com,123-456-7890,10-01-2022,Active,North America,44.0,10-01-2023,09-01-2024,Monthly
2,1,John,Doe,john.doe@example.com,123-456-7890,10-01-2022,Active,North America,101.0,01-07-2023,30-06-2024,Monthly
3,2,Jane,Smith,jane.smith@example.com,098-765-4321,15-12-2021,Inactive,Europe,2.0,15-12-2021,14-12-2022,Monthly
4,2,Jane,Smith,jane.smith@example.com,098-765-4321,15-12-2021,Inactive,Europe,45.0,15-12-2022,14-12-2023,Annual
5,2,Jane,Smith,jane.smith@example.com,098-765-4321,15-12-2021,Inactive,Europe,102.0,15-05-2022,14-05-2023,Annual
6,3,Alice,Johnson,alice.j@example.com,567-890-1234,20-03-2020,Active,Asia,3.0,20-03-2020,19-03-2021,Annual
7,3,Alice,Johnson,alice.j@example.com,567-890-1234,20-03-2020,Active,Asia,46.0,20-03-2021,19-03-2022,Monthly
8,3,Alice,Johnson,alice.j@example.com,567-890-1234,20-03-2020,Active,Asia,103.0,01-08-2021,31-07-2022,Monthly
9,4,Bob,Brown,bob.brown@example.com,234-567-8901,25-06-2019,Inactive,North America,4.0,25-06-2019,24-06-2020,Annual


In [20]:
# Finding total spending of each customer

Transaction['Adjusted_Amount'] = Transaction.apply(
    lambda row: -row['Amount'] if row['TransactionType']== 'Refund'
    else row['Amount'], axis=1)
Transaction.head(10)

,TransactionID,CustomerID,TransactionDate,Amount,TransactionType,Adjusted_Amount
0,1,15,01-01-2024,32,Purchase,32
1,2,22,02-01-2024,52,Refund,-52
2,3,43,03-01-2024,165,Purchase,165
3,4,87,04-01-2024,134,Purchase,134
4,5,34,05-01-2024,158,Purchase,158
5,6,56,06-01-2024,170,Refund,-170
6,7,23,07-01-2024,196,Purchase,196
7,8,11,08-01-2024,195,Purchase,195
8,9,76,09-01-2024,143,Purchase,143
9,10,45,10-01-2024,69,Refund,-69


In [21]:
# Grouping by customerID and sum the Adjusted_Amount

Transaction_summary = Transaction.groupby('CustomerID')['Adjusted_Amount'].sum().reset_index().rename(columns = {'Adjusted_Amount':'CustomerLifetime'})
Transaction_summary.head(10)

,CustomerID,CustomerLifetime
0,11,195
1,15,32
2,17,-73
3,19,-140
4,21,-32
5,22,-52
6,23,196
7,25,-122
8,26,-20
9,28,-153


In [22]:
# Merging the Transaction_summary and then Churn with main dataset df

df = df.merge(Transaction_summary, on='CustomerID', how = 'left')
df.sample(10)

,CustomerID,FirstName,LastName,Email,PhoneNumber,JoinDate,Status,Region,SubscriptionID,StartDate,EndDate,PlanType,CustomerLifetime
352,253,Robert,Jones,robert.jones@example.com,NaN,01-05-2021,Active,Asia,352.0,01-06-2023,01-06-2024,Monthly,114.0
130,46,Frank,Adams,frank.a@example.com,678-901-2345,30-07-2022,Inactive,Asia,88.0,15-05-2021,14-05-2022,Annual,-99.0
282,183,Mia,Young,mia.y@example.com,345-678-9026,14-02-2019,Active,Asia,283.0,15-09-2022,14-09-2023,Annual,-48.0
77,27,Yara,Diaz,yara.d@example.com,567-890-1234,29-09-2020,Active,Asia,27.0,29-09-2020,28-09-2021,Annual,NaN
212,113,Ursula,Green,ursula.g@example.com,345-678-9019,16-04-2019,Active,North America,213.0,15-03-2023,14-03-2024,Monthly,-124.0
106,37,Tom,Wood,tom.w@example.com,987-321-6540,03-03-2023,Active,North America,79.0,03-03-2024,02-03-2025,Annual,143.0
340,241,Sara,Clark,sara.c@example.com,123-456-7910,15-03-2022,Active,Europe,340.0,01-06-2022,01-06-2023,Monthly,132.0
80,28,Zane,Evans,zane.e@example.com,678-901-2345,13-12-2019,Inactive,North America,28.0,13-12-2019,12-12-2020,Annual,-153.0
40,14,Liam,Walker,liam.w@example.com,234-567-8901,08-07-2020,Inactive,Europe,114.0,08-08-2021,07-08-2022,Annual,NaN
350,251,Alice,NaN,alice.j@example.com,123-456-7890,01-01-2023,Active,North America,350.0,01-04-2023,01-04-2024,Monthly,178.0


In [23]:
Churn.head(10)

,ChurnID,CustomerID,ChurnDate,Reason
0,1,22,02-01-2024,Poor Customer Service
1,2,56,06-06-2024,High Prices
2,3,45,10-10-2024,Product Quality Issues
3,4,54,12-12-2024,Lack of Product Features
4,5,82,14-02-2024,Competitor Offerings
5,6,67,15-01-2024,Inconvenience
6,7,17,21-01-2024,Lack of Engagement
7,8,33,18-01-2024,Unmet Expectations
8,9,91,20-01-2024,Price Increases
9,10,23,23-01-2024,Personal Circumstances


In [24]:
df = df.merge(Churn[['CustomerID','ChurnDate']], on = 'CustomerID', how = 'left')
df.sample(5)

,CustomerID,FirstName,LastName,Email,PhoneNumber,JoinDate,Status,Region,SubscriptionID,StartDate,EndDate,PlanType,CustomerLifetime,ChurnDate
285,148,Daisy,Foster,daisy.f@example.com,890-123-4577,10-06-2019,Inactive,Asia,248.0,15-02-2022,14-02-2023,Monthly,-38.0,NaN
264,129,Kevin,Adams,kevin.a@example.com,901-234-5686,18-07-2019,Active,Europe,229.0,15-07-2022,14-07-2023,Annual,-69.0,NaN
88,27,Yara,Diaz,yara.d@example.com,567-890-1234,29-09-2020,Active,Asia,127.0,01-06-2023,31-05-2024,Monthly,NaN,26-02-2024
213,86,Tanya,Brown,tanya.b@example.com,678-901-2349,21-01-2021,Inactive,Asia,186.0,01-12-2022,01-12-2023,Monthly,NaN,NaN
334,196,Zane,Moore,zane.m@example.com,678-901-2360,14-08-2021,Inactive,North America,296.0,15-10-2023,14-10-2024,Monthly,169.0,NaN


In [25]:
df['Churned']= df['ChurnDate'].notnull().astype(int)
df.sample(5)

,CustomerID,FirstName,LastName,Email,PhoneNumber,JoinDate,Status,Region,SubscriptionID,StartDate,EndDate,PlanType,CustomerLifetime,ChurnDate,Churned
286,149,Ethan,Young,ethan.y@example.com,901-234-5688,11-03-2022,Active,North America,249.0,15-03-2022,14-03-2023,Annual,22.0,NaN,0
436,298,Olivia,Perry,olivia.perry@example.com,678-901-2345,12-06-2019,Inactive,Europe,397.0,01-03-2023,01-03-2024,Annual,NaN,NaN,0
310,173,Caleb,Clark,caleb.c@example.com,345-678-9025,31-08-2021,Active,Europe,273.0,01-11-2023,01-11-2024,Annual,118.0,25-04-2024,1
328,190,Tom,Foster,tom.f@example.com,012-345-6793,23-01-2020,Inactive,North America,290.0,15-04-2023,14-04-2024,Monthly,37.0,NaN,0
322,185,Oliver,Davis,oliver.d@example.com,567-890-1248,14-12-2021,Active,Europe,285.0,15-11-2022,14-11-2023,Annual,-149.0,06-05-2024,1


In [26]:
# Cleaning the null values

df['CustomerLifetime']= df['CustomerLifetime'].fillna(0)
df['IsInactive'] = (df['Status']=='Inactive').astype(int)
df.sample(5)

,CustomerID,FirstName,LastName,Email,PhoneNumber,JoinDate,Status,Region,SubscriptionID,StartDate,EndDate,PlanType,CustomerLifetime,ChurnDate,Churned,IsInactive
271,135,Quinn,James,quinn.j@example.com,567-890-1243,30-01-2021,Active,Europe,235.0,15-01-2023,14-01-2024,Annual,-168.0,NaN,0,0
415,277,James,Clark,james.clark@example.com,567-890-1234,15-02-2022,Active,Europe,376.0,01-06-2023,01-06-2024,Monthly,0.0,NaN,0,0
291,154,James,Scott,james.s@example.com,456-789-0134,11-12-2020,Inactive,North America,254.0,01-04-2023,31-03-2024,Annual,153.0,NaN,0,1
142,43,Clara,Bennett,clara.b@example.com,345-678-9012,10-03-2022,Active,Asia,40.0,10-03-2022,31-12-2022,Monthly,165.0,15-01-2024,1,0
27,10,Hank,Wilson,hank.w@example.com,890-123-4567,27-05-2020,Active,North America,52.0,27-05-2021,26-05-2022,Monthly,0.0,NaN,0,0


In [28]:
# Defining Independent(X) and dependent(Y) Variebles

x = df[['CustomerLifetime','IsInactive']]
y = df['Churned']

In [29]:
#Scaling the data
scaler = StandardScaler()

X_scaled = scaler.fit_transform(x)

In [30]:
# Split data for Training and testing (80, 20)

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)

In [31]:
# Train the Logistic Regression model on training data

model = LogisticRegression(max_iter = 1000, C=0.5)
model.fit(x_train, y_train)

LogisticRegression(C=0.5, max_iter=1000)

In [32]:
# Predict the test data

y_pred = model.predict(x_test)

In [33]:
# Evoluate the Model

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
F1 = f1_score(y_test, y_pred)
Confusion = confusion_matrix(y_test, y_pred)


print('Model Evoluation Metrix')
print('-----------------------------------')
print(f'Accuracy: {accuracy:.2f}')
print('Precision', precision)
print('Recall', recall)
print('F1-Score', F1)
print('----------------------------------')
print('Confusion Matrix',Confusion )

Model Evoluation Metrix
-----------------------------------
Accuracy: 0.65
Precision 0.0
Recall 0.0
F1-Score 0.0
----------------------------------
Confusion Matrix [[57  0]
 [31  0]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [34]:
# Loading the new data for prediction

new_data = pd.read_csv('/content/drive/MyDrive/Projects/ML/Sample_Test_Data.csv')
new_data.head(10)

,CustomerLifetime,IsInactive
0,102,1
1,435,1
2,860,1
3,270,1
4,106,1
5,71,1
6,700,0
7,20,1
8,614,0
9,121,1


In [35]:
# Applying the trained data

new_data['Predicted_Churn']= model.predict(new_data)
new_data.head(10)

,CustomerLifetime,IsInactive,Predicted_Churn
0,102,1,0
1,435,1,0
2,860,1,0
3,270,1,0
4,106,1,0
5,71,1,0
6,700,0,0
7,20,1,0
8,614,0,0
9,121,1,0


In [36]:
new_data['Predicted_Churn'] = new_data['Predicted_Churn'].map({1:'Churn' , 0:'Not Churned'})
new_data.sample(20)

,CustomerLifetime,IsInactive,Predicted_Churn
17,871,1,Not Churned
21,308,0,Not Churned
33,313,0,Not Churned
52,875,1,Not Churned
46,782,1,Not Churned
18,663,0,Not Churned
13,458,0,Not Churned
96,476,0,Not Churned
63,166,1,Not Churned
68,13,0,Not Churned


In [37]:
new_data.to_csv('Churned Model.csv', index = False)

In [38]:
# In this project, I built a Logistic Regression model to predict whether a Customer churned or not based on their spendings and platform activities.
# The workflow included data loading, preprocessing, encoding categorical variables using one-hot encoding,
# splitting the dataset into training and testing sets, and training a classification model using Scikit-Learn.

# To evaluate model performance, I used metrics such as Accuracy, Precision, Recall, and F1-Score,
# which provided a well-rounded understanding of how effectively the model classified churning outcomes.
# I also applied the model to new, unseen data, generating predictions and exporting the results for practical use.

# This project helped me understand the machine learning pipeline—from raw data to deployable predictions.
# It strengthened my foundations in data preprocessing, model training, and evaluation.
# Future improvements could include alternate and complex classification algorithms.